In [2]:
#1. load PDF using langchain document loader
from langchain_community.document_loaders import PyPDFLoader

pdfLoader=PyPDFLoader('PY2026 QHP Issuer Instructions_AppE_NA_v1.pdf')
pdfDocs=pdfLoader.load()
pdfDocs

[Document(metadata={'producer': 'Adobe PDF Library 25.1.211', 'creator': 'Acrobat PDFMaker 25 for Word', 'creationdate': '2025-04-08T22:25:52-04:00', 'author': 'CMS', 'base target': '_self', 'comments': '', 'company': 'CMS', 'contenttypeid': '0x0101006E134D1240AD6D4EA01100BFA58605E2', 'mediaserviceimagetags': '', 'moddate': '2025-04-09T08:23:19-04:00', 'nccl_app': 'PDF', 'nccl_standard': 'WCAG 2.0 AA;WCAG 2.1 AA;', 'nccl_status': 'Passed', 'sourcemodified': '', 'subject': 'Qualified Health Plan Issuer Application Instructions PY2026', 'title': 'Extracted Appendix E: Network Adequacy', 'source': 'PY2026 QHP Issuer Instructions_AppE_NA_v1.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='Qualified Health Plan  \nIssuer Application Instructions  \nPlan Year 2026 \n \n \nExtracted: \nAppendix E: Network Adequacy Standards'),
 Document(metadata={'producer': 'Adobe PDF Library 25.1.211', 'creator': 'Acrobat PDFMaker 25 for Word', 'creationdate': '2025-04-08T22:25:52-04:00'

In [3]:
#2. semantic chunking
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

openAIembeddings = OpenAIEmbeddings(model="text-embedding-3-small")
openAIembeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x116090c20>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x116091550>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [4]:
#convert list of documents to list of strings
from typing import List

def convert_docs_to_strings(docs: List["Document"]) -> List[str]:
    """Converts a list of LangChain Documents to a list of strings.

    Args:
        docs: A list of LangChain Document objects.

    Returns:
        A list of strings containing the page_content of each Document.
    """
    return [doc.page_content for doc in docs]

In [5]:
pdfStringlist=convert_docs_to_strings(pdfDocs)

In [6]:
from langchain_experimental.text_splitter import SemanticChunker


semanticTextSplitter = SemanticChunker(openAIembeddings)
semanticPdfSplitDocs = semanticTextSplitter.create_documents(pdfStringlist)
print(semanticPdfSplitDocs[2].page_content)

Table E-1. Time and Distance Standards for Individual Provider Specialty Types for Medical QHPs  
for Exchange PY2026 QHP Certification 
Individual Provider Specialty Types 
Maximum Time and Distance Standardsa 
Large Metro 
County Metro County Micro County Rural County 
Counties with Extreme 
Access Considerations 
(CEAC) 
Time Distance Time Distance Time Distance Time Distance Time Distance 
Allergy and Immunology 30 15 45 30 80 60 90 75 125 110 
Cardiology 20 10 30 20 50 35 75 60 95 85 
Cardiothoracic Surgery 30 15 60 40 100 75 110 90 145 130 
Chiropractor 30 15 45 30 80 60 90 75 125 110 
Dental 30 15 45 30 80 60 90 75 125 110 
Dermatology 20 10 45 30 60 45 75 60 110 100 
Emergency Medicine 20 10 45 30 80 60 75 60 110 100 
Endocrinology 30 15 60 40 100 75 110 90 145 130 
ENT/Otolaryngology 30 15 45 30 80 60 90 75 125 110 
Gastroenterology 20 10 45 30 60 45 75 60 110 100 
General Surgery 20 10 30 20 50 35 75 60 95 85 
Gynecology, OB/GYN 10 5 15 10 30 20 40 30 70 60 
Infectious Diseas

In [ ]:
# 3 use astra db vector db
#astra db
from langchain_astradb import AstraDBVectorStore
import os
from dotenv import load_dotenv

load_dotenv()
ASTRA_DB_API_ENDPOINT=os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN=os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_KEYSPACE=os.getenv("ASTRA_DB_KEYSPACE")


def ingestdata(status):
    astraDBvstore=AstraDBVectorStore(
        embedding=openAIembeddings,
        collection_name="assignment1_1",
        api_endpoint=ASTRA_DB_API_ENDPOINT,
        token=ASTRA_DB_APPLICATION_TOKEN,
        namespace=ASTRA_DB_KEYSPACE,
    )

    storage=status

    if storage==None:
        inserted_ids=astraDBvstore.add_documents(semanticPdfSplitDocs)
    else:
        return astraDBvstore
    return astraDBvstore,inserted_ids

astraDBvstore,inserted_ids=ingestdata(None) 
print(f"\n Inserted {len(inserted_ids)} documents.")   
results= astraDBvstore.similarity_search("what are individual provider specialty types?")
results


 Inserted 7 documents.


[Document(id='cd110a9053774f7a8bc905bedb853364', metadata={}, page_content='Table E-1. Time and Distance Standards for Individual Provider Specialty Types for Medical QHPs  \nfor Exchange PY2026 QHP Certification \nIndividual Provider Specialty Types \nMaximum Time and Distance Standardsa \nLarge Metro \nCounty Metro County Micro County Rural County \nCounties with Extreme \nAccess Considerations \n(CEAC) \nTime Distance Time Distance Time Distance Time Distance Time Distance \nAllergy and Immunology 30 15 45 30 80 60 90 75 125 110 \nCardiology 20 10 30 20 50 35 75 60 95 85 \nCardiothoracic Surgery 30 15 60 40 100 75 110 90 145 130 \nChiropractor 30 15 45 30 80 60 90 75 125 110 \nDental 30 15 45 30 80 60 90 75 125 110 \nDermatology 20 10 45 30 60 45 75 60 110 100 \nEmergency Medicine 20 10 45 30 80 60 75 60 110 100 \nEndocrinology 30 15 60 40 100 75 110 90 145 130 \nENT/Otolaryngology 30 15 45 30 80 60 90 75 125 110 \nGastroenterology 20 10 45 30 60 45 75 60 110 100 \nGeneral Surgery 2

In [13]:
#try all index types using pinecone db
#flat index
from pinecone import Pinecone
import os
from dotenv import load_dotenv
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore

load_dotenv()

pinecone_api_key=os.getenv("PINECONE_API_KEY")
pineConeDB=Pinecone(api_key=pinecone_api_key)



index_name="assignment12"

if not pineConeDB.has_index(index_name):
    pineConeDB.create_index(
        name=index_name,
        dimension=1536, # open ai  model dimension
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

pineConeIndex=pineConeDB.Index(index_name)


vector_store= PineconeVectorStore(index=pineConeIndex,embedding=openAIembeddings)

In [14]:
#create unique ids
from uuid import uuid4
uuids=[str(uuid4()) for _ in range(len(semanticPdfSplitDocs))]

In [15]:
#add pdf to vector store - using chunked split docs
vector_store.add_documents(documents=semanticPdfSplitDocs, ids=uuids) #4 docs split into 12 added in vector db

['b657c3ed-d157-4b4a-9ad5-048272985ac9',
 '4b4d5b1e-64e8-4607-b929-a8b5bf47b8c1',
 '59027d75-cd83-43c9-acac-ece7737f7fa2',
 '483dd208-e857-42aa-a0ee-57e80338a145',
 '815c2708-fbc2-4cda-b545-3bd080480538',
 '375e67d2-c8b7-40c8-abe9-07f2a5107a75',
 '0fc44e38-26ac-4e71-8db6-f2c06ae7475f']

In [16]:
results=vector_store.similarity_search("wwhat are individual provider specialty types?", k=1) # will use cosine similarity
results

[Document(id='59027d75-cd83-43c9-acac-ece7737f7fa2', metadata={}, page_content='Table E-1. Time and Distance Standards for Individual Provider Specialty Types for Medical QHPs  \nfor Exchange PY2026 QHP Certification \nIndividual Provider Specialty Types \nMaximum Time and Distance Standardsa \nLarge Metro \nCounty Metro County Micro County Rural County \nCounties with Extreme \nAccess Considerations \n(CEAC) \nTime Distance Time Distance Time Distance Time Distance Time Distance \nAllergy and Immunology 30 15 45 30 80 60 90 75 125 110 \nCardiology 20 10 30 20 50 35 75 60 95 85 \nCardiothoracic Surgery 30 15 60 40 100 75 110 90 145 130 \nChiropractor 30 15 45 30 80 60 90 75 125 110 \nDental 30 15 45 30 80 60 90 75 125 110 \nDermatology 20 10 45 30 60 45 75 60 110 100 \nEmergency Medicine 20 10 45 30 80 60 75 60 110 100 \nEndocrinology 30 15 60 40 100 75 110 90 145 130 \nENT/Otolaryngology 30 15 45 30 80 60 90 75 125 110 \nGastroenterology 20 10 45 30 60 45 75 60 110 100 \nGeneral Surge

In [ ]:
#make a retriever so u can use in RAG
retriever=vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold":0.8} #i need the exact page that has table, if table is split into 2 pages, give both pages
)

In [20]:
retriever.invoke("what is time and distance rule for cardiology large metro?")

No relevant docs were retrieved using the relevance score threshold 0.8


[]

In [21]:
# make model for RAG

from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [22]:
# create prompt

from langchain_core.prompts import PromptTemplate

prompt=PromptTemplate(
    template="""You are a CMS Network Adequacy Auditor. You know all CMS Rules for Network Adequacy.
    Go through retrieved context to answer questions from latest CMS QHP filling Instructions.
    If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.""",
    input_variables=['context','question']
)

In [23]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [24]:
#output parser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [25]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [26]:
rag_chain.invoke("what are individual provider specialty types?")

No relevant docs were retrieved using the relevance score threshold 0.8


"Okay, I'm ready. Please provide the retrieved context and the questions from the latest CMS QHP filing instructions.  I will do my best to answer them concisely based on my knowledge of CMS network adequacy rules.  I will only respond if I have sufficient information; otherwise, I will state that I don't know."

In [37]:
llm_output=rag_chain.invoke("what is time and distance rule for cardiology large metro?")

No relevant docs were retrieved using the relevance score threshold 0.8


In [38]:


# Write to a new file
with open("output.txt", "w") as file:
    file.write(llm_output)
